In [7]:
import csv
import pandas as pd
import os
import time

from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import NoSuchElementException
from selenium.common.exceptions import TimeoutException
from selenium.common.exceptions import StaleElementReferenceException
from selenium.webdriver.common.by import By 
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

############################################################################################
#Initial variables
#This part of the code is used to set the inital parameters for the web crawler
############################################################################################
START_PAGE = 1
END_PAGE = 5
PROJECT_ABBR = "hktutor"
PROJECT_NAME = "Project 5"
URL = "http://www.hktutor.hk/index.php?pageNum=1"

#Main window
driver =webdriver.Firefox(executable_path=r'D:\git\FINA2390-Programs\geckodriver.exe')
driver.get(URL)

df_columns = ['Date',
              'Year of Study',
              'Subject',
              'Sex',
              'Location',
              'Time',
              'Salary',
              'Remarks']


df_etf = pd.DataFrame(columns=df_columns)


def go_to_page(page):
    if page != 1:
        print 'GOING TO PAGE' + str(page)
        driver.get('http://www.hktutor.hk/index.php?pageNum='+str(page))
        print 'ARRIVED PAGE' + str(page)
    else:
        return

def save_file(state, CURRENT_PAGE):
    if state == 'backup':
        filename = '['+ PROJECT_ABBR +'][Intrim Backup][' + str(CURRENT_PAGE) + ']' + PROJECT_NAME + ' PAGE ' + str(START_PAGE) + ' to PAGE ' + str(CURRENT_PAGE) + ' Records.csv'
        df_etf.to_csv(filename, index=False,  encoding='utf_8_sig')
        print '[Intrim Backup]Saved First '+ str(CURRENT_PAGE) + 'Pages Records to CSV'
    elif state == 'all':
        filename = '['+ PROJECT_ABBR +']'+ PROJECT_NAME +' PAGE ' + str(START_PAGE) + 'to PAGE ' + str(END_PAGE) + 'Records.csv'
        df_etf.to_csv(filename, index=False, encoding='utf_8_sig')
        print 'Exported to csv'
        
                                                

############################################################################################
#Main
#This part is the main function of the program
############################################################################################
def main():
    time.sleep(2)
    
    go_to_page(START_PAGE)
    time.sleep(1)
    print 'Start at PAGE'+ str(START_PAGE)
    
    n = 1
    for CURRENT_PAGE in range(START_PAGE,END_PAGE+1):
        print '--------------------PAGE ' + str(CURRENT_PAGE) +' START--------------------'
        START_TIMESTAMP = time.time()
        
        row_count = len(driver.find_elements_by_xpath('//*[@id="mid"]/div[2]/div[2]/div[2]/table'))
        
        for row in driver.find_elements_by_xpath('//*[@id="right_content"]/div[1]/div/table/tbody'):
            df_etf.loc[n, 'Location']         = row.find_element_by_xpath('tr[1]/td[1]').text
            df_etf.loc[n, 'Date']             = row.find_element_by_xpath('tr[1]/td[4]').text
            df_etf.loc[n, 'Subject']          = row.find_element_by_xpath('tr[2]/td[1]').text
            df_etf.loc[n, 'Time']             = row.find_element_by_xpath('tr[3]/td[2]').text
            df_etf.loc[n, 'Salary']           = row.find_element_by_xpath('tr[3]/td[1]/span').text
            df_etf.loc[n, 'Remarks']          = row.find_element_by_xpath('tr[4]/td[1]').text.split('\n')[0]
            
            sex_yof = row.find_element_by_xpath('tr[2]/td[2]').text.split('(')
            df_etf.loc[n, 'Year of Study']    = sex_yof[0]
            df_etf.loc[n, 'Sex']              = sex_yof[1].split(')')[0]
            
            print '--------------------ROW ' + str(n) +' FINISHED--------------------'
            n = n + 1
            
        print '--------------------PAGE ' + str(CURRENT_PAGE) +' FINISH--------------------'
        END_TIMESTAMP = time.time()
        print '-TIME USED:' + str(START_TIMESTAMP-END_TIMESTAMP) + '-'
        
        #store every first 5,10,15... pages to CSV for backup purpose
        if(CURRENT_PAGE % 5) == 0:
            save_file('backup', CURRENT_PAGE)
            
        go_to_page(CURRENT_PAGE + 1)
        
    #set Source as QS for every row as this is crawl from QS
    
    #export the file to CSV
    save_file('all', CURRENT_PAGE)
    
    #close the driver
    driver.quit()
    
if __name__ == "__main__":
    main()



Start at PAGE1
--------------------PAGE 1 START--------------------
--------------------ROW 1 FINISHED--------------------
--------------------ROW 2 FINISHED--------------------
--------------------ROW 3 FINISHED--------------------
--------------------ROW 4 FINISHED--------------------
--------------------ROW 5 FINISHED--------------------
--------------------ROW 6 FINISHED--------------------
--------------------ROW 7 FINISHED--------------------
--------------------ROW 8 FINISHED--------------------
--------------------ROW 9 FINISHED--------------------
--------------------ROW 10 FINISHED--------------------
--------------------ROW 11 FINISHED--------------------
--------------------ROW 12 FINISHED--------------------
--------------------ROW 13 FINISHED--------------------
--------------------ROW 14 FINISHED--------------------
--------------------ROW 15 FINISHED--------------------
--------------------PAGE 1 FINISH--------------------
-TIME USED:-1.51900005341-
GOING TO PAGE2
ARRIV

In [8]:
df_etf


,Date,Year of Study,Subject,Sex,Location,Time,Salary,Remarks
1,4小時前,中三,"英文,數學,會計",女,將軍澳,"1 堂 x 1.5小時, 星期五",$140,女老師 5PM
2,6小時前,小一,普通話,男,西營盤,"1 堂 x 45分鐘, 星期六",$150,六：全日，需考獲演奏級，有經驗優先，大學生都可，女導師
3,6小時前,小一,普通話,男,西營盤,"1 堂 x 1小時, 星期一至六",$110,平日：4：30PM後，五：3PM後，六：全日，內地來港／有水平測試證書女大學生
4,12小時前,小五,"全科,英文,常識",女,荃灣,"2 堂 x 2小時, 星期一至日",$120,平日：4：30PM後，六日：全日，1-2堂，特殊學校學生，認知比較差，反應較慢，學習形式無所...
5,12小時前,小一,"全科,中文,英文",男,荃灣,"2 堂 x 1.5小時, 星期三六日",$100,三：六日：全日，跟功課及溫習，女導師
6,15小時前,小四,全科,男,將軍澳坑口,"4 堂 x 1小時, 星期一二三四",$120,平日：5-5：30PM，男女導師
7,17小時前,中二,全科,男,將軍澳,"3 堂 x 1.5小時, 星期一三五",$120,一三五：5:30pm，需大學或以上程度，中文科L5
8,1日前,小一,全科,女,葵涌,"2 堂 x 1.5小時, 星期三四",$110,"5:15PM,女導師，英文DSE L4以上"
9,1日前,小五,"英文,數學",男,大圍,"1 堂 x 1小時, 星期五",$100,5pm
10,1日前,小三,英文,男,將軍澳坑口,"2 堂 x 1.5小時, 星期一四",$100,"一四：4-5pm,女導師"
